# Weather

In [9]:
# API documentation: https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/#elementor-toc__heading-anchor-8

# 1. Import packages
import requests
import pandas as pd
import time
from datetime import date
import os
from dotenv import load_dotenv

pd.set_option('display.max_columns', None)

In [2]:
# 2. Extract API key
load_dotenv(dotenv_path="../config/.env")
api_key = os.getenv("weather_api_key")

print(api_key)

S3QJE5ZYXMHKHXN5LWH5MP9MX


In [19]:
# 3. Define main path parameters (location and date) and compile URL
# Base format for visual crossing: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/[location]/[date1]/[date2]?key=YOUR_API_KEY
        
# Select location
location_dictionary = {
    "Lake Arrowhead": "34.250470%2c-117.189692",
    "Crestline" : "34.241838,-117.280022",
    "Running Springs" : "34.208097,-117.089241",
    "Big Bear" : "34.240315,-116.911361"
}

location = "Lake Arrowhead"

# Select dates yyyy-MM-dd
#/timeline/[location] – 15-day forecast
#/timeline/[location]/[date1] – query for date 1 only
#/timeline/[location]/[date1]/[date2] – queries for a specific date range

date_1 = "2023-01-01"
date_2 = "2023-12-31"

# Compile URL
base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
compiled_url = f"{base_url}{location_dictionary[location]}/{date_1}/{date_2}"

print(compiled_url)

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/34.250470%2c-117.189692/2023-01-01/2023-12-31


In [20]:
# 4. Define additional query parameters
elements = ["conditions", "description", "source"]
join_char = ","
elements_string = join_char.join(elements)

params = {
    "key": api_key, 
    "unitGroup": "us",
    "include": "days",
    "contentType" : "json"
}

In [ ]:
# 5. Define headers. Not needed here because key is in the URL. Included for completeness.
# headers = {
#    "Authorization": api_key, 
#    "key 2": "value 2", 
#    "key 3": "value 3", 
#   }

In [ ]:
# 6. Single API request function plus error handling

def get_single_page(url, params, headers = None):
    try:
        response = requests.get(compiled_url, params = params, headers = headers, timeout = 10)
        response.raise_for_status()         # If an error occurs, this returns an HTTPError object with more error details
        return response.json()
    except requests.exceptions.HTTPError as e:
        try:
            error_details = response.json()         # Try parsing as JSON
        except ValueError:
            error_details = response.text           # Fallback to raw text
        print(f"HTTP error occurred: {e}")
        print(f"Response content: {error_details}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    

{'queryCost': 146, 'latitude': 34.25047, 'longitude': -117.189692, 'resolvedAddress': '34.250470,-117.189692', 'address': '34.250470,-117.189692', 'timezone': 'America/Los_Angeles', 'tzoffset': -8.0, 'days': [{'datetime': '2025-01-01', 'datetimeEpoch': 1735718400, 'tempmax': 66.0, 'tempmin': 30.5, 'temp': 45.2, 'feelslikemax': 66.0, 'feelslikemin': 29.6, 'feelslike': 44.9, 'dew': 16.9, 'humidity': 35.5, 'precip': 0.0, 'precipprob': 0.0, 'precipcover': 0.0, 'preciptype': None, 'snow': 0.0, 'snowdepth': 0.0, 'windgust': 19.7, 'windspeed': 8.6, 'winddir': 316.6, 'pressure': 1021.8, 'cloudcover': 0.0, 'visibility': 9.3, 'solarradiation': 132.8, 'solarenergy': 11.3, 'uvindex': 6.0, 'severerisk': 10.0, 'sunrise': '06:55:06', 'sunriseEpoch': 1735743306, 'sunset': '16:50:19', 'sunsetEpoch': 1735779019, 'moonphase': 0.07, 'conditions': 'Clear', 'description': 'Clear conditions throughout the day.', 'icon': 'clear-day', 'stations': ['KL35', 'KRAL', '72286023119', 'A0685400115', 'SE372', '7228690

In [ ]:
# 7. Function to loop over pages and compile data

def get_all_pages(base_url, base_params, headers = None, limit = 100):
    all_results = []
    page = 1

    while True:
        params = base_params.copy()     # Feed a fresh copy of the params into the loop each time
        params["page"] = page           # Add a page parameter to the params dictionary
        params["limit"] = limit         # Add a limit parameter to the params dictionary

        data = get_single_page(base_url, params, headers)           # Get 1 page of data.
        if not data or "results" not in data:
            break

        results = data["days"]          # The response (here named "data") is a dictionary with nested data
        all_results.extend(results)     # For Visual Crossing we only need the "days" (key) values

        if len(results) < limit:        # If "days" contains fewer items than our limit, we know we've reached the end
            break

        page += 1
        time.sleep(1)

    return all_results

In [21]:
weather_data = get_single_page(compiled_url, params)

HTTP error occurred: 429 Client Error:  for url: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/34.250470%2C-117.189692/2023-01-01/2023-12-31?key=S3QJE5ZYXMHKHXN5LWH5MP9MX&unitGroup=us&include=days&contentType=json
Response content: You have exceeded the maximum number of daily result records for your account. 


In [17]:
#6	Export / Load - Save or return clean dataset
if weather_data:
    try:
        df = pd.DataFrame(weather_data["days"])  # Adjust this based on API structure
        print(df.head())
    except Exception as e:
        print("Could not convert to DataFrame:", e)

     datetime  datetimeEpoch  tempmax  tempmin  temp  feelslikemax  \
0  2024-01-01     1704096000     47.1     26.8  35.6          44.6   
1  2024-01-02     1704182400     51.6     22.9  36.4          51.6   
2  2024-01-03     1704268800     41.8     32.7  37.8          39.4   
3  2024-01-04     1704355200     45.0     27.9  35.9          42.9   
4  2024-01-05     1704441600     46.3     25.0  35.2          43.2   

   feelslikemin  feelslike   dew  humidity  precip  precipprob  precipcover  \
0          26.8       34.4  24.6      66.8   0.000         0.0         0.00   
1          22.9       35.9  24.3      65.0   0.000         0.0         0.00   
2          24.7       32.5  31.1      77.2   0.155       100.0        16.67   
3          23.3       33.2  22.6      61.1   0.000         0.0         0.00   
4          25.0       33.2  21.6      60.4   0.000         0.0         0.00   

  preciptype  snow  snowdepth  windgust  windspeed  winddir  pressure  \
0       None   0.0        0.0  

In [11]:
df

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,tzoffset
0,2025-01-01,1735718400,66.0,30.5,45.2,66.0,29.6,44.9,16.9,35.5,0.0,0.0,0.0,None,0.0,0.0,19.7,8.6,316.6,1021.8,0.0,9.3,132.8,11.3,6.0,10.0,06:55:06,1735743306,16:50:19,1735779019,0.07,Clear,Clear conditions throughout the day.,clear-day,"[KL35, KRAL, 72286023119, A0685400115, SE372, ...",obs,NaN
1,2025-01-02,1735804800,72.0,31.5,47.6,72.0,31.5,47.4,16.4,32.8,0.0,0.0,0.0,None,0.0,0.0,14.5,8.8,352.1,1022.5,0.2,9.9,135.2,11.5,6.0,10.0,06:55:16,1735829716,16:51:06,1735865466,0.10,Clear,Clear conditions throughout the day.,clear-day,"[KL35, KRAL, 72286023119, KRIV, A0685400115, S...",obs,NaN
2,2025-01-03,1735891200,62.4,40.7,49.9,62.4,38.4,48.6,11.6,22.7,0.0,0.0,0.0,None,0.0,0.0,19.9,10.8,291.7,1019.8,0.6,9.6,103.5,8.9,5.0,10.0,06:55:24,1735916124,16:51:53,1735951913,0.14,Clear,Clear conditions throughout the day.,clear-day,"[KL35, KRAL, 72286023119, A0685400115, SE372, ...",obs,NaN
3,2025-01-04,1735977600,52.0,36.6,42.8,52.0,32.9,41.1,24.3,49.1,0.0,0.0,0.0,[rain],0.0,0.0,21.9,8.4,2.1,1019.8,18.9,7.9,124.0,10.7,6.0,10.0,06:55:30,1736002530,16:52:42,1736038362,0.17,Clear,Clear conditions throughout the day.,clear-day,"[KL35, KRAL, 72286023119, KRIV, A0685400115, S...",obs,NaN
4,2025-01-05,1736064000,61.9,33.0,45.0,61.9,29.8,44.8,21.3,40.6,0.0,0.0,0.0,None,0.0,0.0,16.3,7.9,34.4,1022.9,0.0,9.8,118.0,10.3,5.0,10.0,06:55:34,1736088934,16:53:32,1736124812,0.21,Clear,Clear conditions throughout the day.,clear-day,"[KL35, KRAL, 72286023119, A0685400115, SE372, ...",obs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2025-05-22,1747897200,80.2,45.4,64.5,80.0,45.4,64.4,35.7,39.2,0.0,0.0,0.0,None,0.0,0.0,24.2,14.5,288.1,1016.9,0.6,9.9,260.0,22.4,10.0,10.0,05:42:06,1747917726,19:49:22,1747968562,0.85,Clear,Clear conditions throughout the day.,clear-day,"[KL35, KRAL, 72286023119, KRIV, A0685400115, S...",obs,-7.0
142,2025-05-23,1747983600,80.3,46.4,61.7,80.2,46.3,61.7,38.9,48.1,0.0,0.0,0.0,[rain],0.0,0.0,21.7,15.4,277.7,1015.9,7.4,9.4,261.8,22.7,10.0,10.0,05:41:34,1748004094,19:50:04,1748055004,0.88,Clear,Clear conditions throughout the day.,clear-day,"[KL35, KRAL, 72286023119, KRIV, SE372, 7228690...",obs,-7.0
143,2025-05-24,1748070000,70.6,49.7,59.9,70.6,49.7,59.9,36.1,42.8,0.0,0.0,0.0,[rain],0.0,0.0,24.2,11.0,260.5,1016.6,25.2,9.9,248.8,21.6,10.0,10.0,05:41:04,1748090464,19:50:46,1748141446,0.92,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KL35, KRAL, KRIV, SE372]",obs,-7.0
144,2025-05-25,1748156400,74.0,45.2,58.6,74.0,45.2,58.6,39.4,52.5,0.0,0.0,0.0,[rain],0.0,0.0,25.3,11.8,316.0,1018.2,21.7,9.9,276.4,23.9,10.0,10.0,05:40:35,1748176835,19:51:26,1748227886,0.95,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KL35, KRAL, KRIV, SE372]",obs,-7.0


In [18]:
csv_file = "weather_lake_arrowhead_raw"

df.to_csv(csv_file, mode = "a", index = False, header = False)